In [1]:
#@markdown # Download repository
!git clone 'https://github.com/DevashishPrasad/Virtual-AI-assistant'

Cloning into 'Virtual-AI-assistant'...
remote: Enumerating objects: 2860, done.
remote: Counting objects: 100% (2860/2860), done.
remote: Compressing objects: 100% (1177/1177), done.
remote: Total 2860 (delta 1644), reused 2857 (delta 1644), pack-reused 0
Receiving objects: 100% (2860/2860), 21.15 MiB | 17.78 MiB/s, done.
Resolving deltas: 100% (1644/1644), done.


In [ ]:
#@markdown # Installations

!pip install -qq ffmpeg-python
!pip install -qq SpeechRecognition
!pip install -qq pyttsx3
!pip install -qq sounddevice
!sudo apt-get install -qq libportaudio2
!pip install -qq librosa unidecode gdown
!pip install -qq -r "/content/Virtual-AI-assistant/code/closed_domain/JointBERT/requirements.txt"

     |████████████████████████████████| 32.8 MB 38 kB/s 


In [ ]:
#@markdown # Download Models

# flowtron models
!gdown https://drive.google.com/uc?id=1KhJcPawFgmfvwV7tQAOeC253rYstLrs8
!gdown https://drive.google.com/uc?id=1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF
# joint bert model
!gdown https://drive.google.com/uc?id=16pckkEr44qI78x-Yuymvv2Htx6U5gaM9

# paste model at their locations
!mv '/content/waveglow_256channels_universal_v5.pt' '/content/Virtual-AI-assistant/code/flowtron/waveglow_256channels_universal_v5.pt'
!mv '/content/flowtron_libritts.pt' '/content/Virtual-AI-assistant/code/flowtron/flowtron_libritts.pt'
!mv '/content/pytorch_model.bin' '/content/Virtual-AI-assistant/code/closed_domain/JointBERT/atis_model/pytorch_model.bin'

!mv '/content/Virtual-AI-assistant/code/flowtron/data/cmudict_dictionary.txt' '/content/Virtual-AI-assistant/code/flowtron/data/cmudict_dictionary'

In [ ]:
#@markdown # Setup Code
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio, display
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read, write
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def save_audio(file_name):
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = read(io.BytesIO(riff))
  write(file_name, sr, audio)

%cd "/content/Virtual-AI-assistant/code"

import warnings 
warnings.filterwarnings("ignore")
from IPython.display import clear_output
import speech_recognition as sr 
import time
import os
os.environ['LRU_CACHE_CAPACITY'] = '1'
import pyttsx3  
import sys  
sys.path.insert(0, '/content/Virtual-AI-assistant/code/flowtron')
sys.path.insert(0, '/content/Virtual-AI-assistant/code/closed_domain')
import torch, gc
import flowAPI
from flowAPI import FlowAPI
from closed_domain.IntentDemo import IntentReply
from IPython.display import HTML
from base64 import b64encode

class AI:
    def __init__(self):
        gc.collect()
        torch.cuda.empty_cache()
        self.r = sr.Recognizer() 
        self.fp = flowAPI.FlowAPI()
        self.query = ""
        self.resp = ""
        self.responder = IntentReply()

    def listen(self):
        try:
            # use the microphone as source for input
            audioFile = 'test.wav'
            save_audio(audioFile)
            userAudio = sr.AudioFile(audioFile)
            with userAudio as source2:   
                #listens for the user's input  
                audio2 = self.r.record(source2)
                # Using google to recognize audio 
                query = self.r.recognize_google(audio2) 
                query = query.lower() 
                clear_output()
                print("You : ",query)
                self.query = query
                print("[INFO] : Thinking, PLease wait")

        except sr.RequestError as e: 
            print("Could not request results; {0}".format(e)) 
              
        except sr.UnknownValueError: 
            print("unknown error occured")
        finally:
        	pass

    def respond(self):
        self.resp = self.responder(self.query)

    def speak(self,SPEAKER_ID = 196):
        # 196 is a male voice & 1246 is a female voice
        self.fp.synthesize(1246,self.resp)
        torch.cuda.empty_cache()

    # available speaker ids: 1069, 1088, 1116, 118, 1246, 125, 1263, 1502, 1578, 1841, 1867, 196, 1963, 1970, 200, 2092, 2136, 2182, 2196, 2289, 2416, 2436, 250, 254, 2836, 2843, 2911, 2952, 3240, 3242, 3259, 3436, 3486, 3526, 3664, 374, 3857, 3879, 3982, 3983, 40, 4018, 405, 4051, 4088, 4160, 4195, 4267, 4297, 4362, 4397, 4406, 446, 460, 4640, 4680, 4788, 5022, 5104, 5322, 5339, 5393, 5652, 5678, 5703, 5750, 5808, 587, 6019, 6064, 6078, 6081, 6147, 6181, 6209, 6272, 6367, 6385, 6415, 6437, 6454, 6476, 6529, 669, 6818, 6836, 6848, 696, 7059, 7067, 7078, 7178, 7190, 7226, 7278, 730, 7302, 7367, 7402, 7447, 7505, 7511, 7794, 78, 7800, 8051, 8088, 8098, 8108, 8123, 8238, 83, 831, 8312, 8324, 8419, 8468, 8609, 8629, 87, 8770, 8838, 887

A = AI()
clear_output()

In [ ]:
#@markdown # Engine

A.listen()
A.respond()
A.speak()

!rm "/content/Virtual-AI-assistant/code/output2.mp4"
!ffmpeg -i "/content/Virtual-AI-assistant/code/output.mp4" -filter:a "volume=10.0" -vcodec libx264 -acodec aac "/content/Virtual-AI-assistant/code/output2.mp4" 2> /dev/null

mp4 = open('/content/Virtual-AI-assistant/code/output2.mp4','rb').read()
!rm "/content/Virtual-AI-assistant/code/output.mp4"
!rm "/content/Virtual-AI-assistant/code/my_audio.mp4"
!rm "/content/Virtual-AI-assistant/code/my_video.mp4"
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# hello mumbai to pune
# hello can you give me minimum fare flights from nagpur to pune business class
# hello can you give me air india flights from nagpur to pune
# show me flight with id 9
# hello how many seats are available from nagpur to pune